In [3]:
import numpy as np
import os

DIR = os.path.join('./')
animal = 'MD589'
volume_path = os.path.join(DIR, f'{animal}_annotations.npy')

# Numpy array whose shape should match (x, y, z)
with open(volume_path, 'rb') as file:
    volume = np.load(file)
volume = np.swapaxes(volume, 0, 1)

# Voxel resolution in nanometer (how much nanometer each element in numpy array represent)
resol = (14464, 14464, 20000)

# Voxel offset
offset = (0, 0, 0)

# Layer type
layer_type = ['segmentation'][0]

In [4]:
# Run this cell if you want to preview the numpy array before converting to precomputed format
import neuroglancer
viewer = neuroglancer.Viewer()
print(viewer)

all_volume_layer = neuroglancer.SegmentationLayer(
    source = neuroglancer.LocalVolume(
        data=volume, 
        dimensions=neuroglancer.CoordinateSpace(names=['x', 'y', 'z'], units='nm', scales=resol), 
        voxel_offset=offset
    ),
)

with viewer.txn() as s:
    s.layers.clear()
    s.layers['all'] = all_volume_layer

http://127.0.0.1:36105/v/435881137d12780cb2a2edacdb2cfc43eab8301d/


In [5]:
# https://github.com/seung-lab/cloud-volume
from cloudvolume import CloudVolume

precompute_path = os.path.join(DIR, f'{animal}_annotations')
cloudpath = f'file://{precompute_path}'
info = CloudVolume.create_new_info(
    num_channels = 1,
    layer_type = layer_type,
    data_type = str(volume.dtype), # Channel images might be 'uint8'
    encoding = 'raw', # raw, jpeg, compressed_segmentation, fpzip, kempressed
    resolution = resol, # Voxel scaling, units are in nanometers
    voxel_offset = offset, # x,y,z offset in voxels from the origin
    chunk_size = [128, 128, 16], # units are voxels
    volume_size = volume.shape, # e.g. a cubic millimeter dataset
)
vol = CloudVolume(cloudpath, mip=0, info=info, compress=False)
vol.info['segment_properties'] = 'names'
vol.commit_info()
vol[:, :, :] = volume[:, :, :]

Uploading: 100%|██████████| 2464/2464 [00:12<00:00, 15.32it/s] 

In [6]:
# https://github.com/seung-lab/igneous
from taskqueue import LocalTaskQueue
import igneous.task_creation as tc

tq = LocalTaskQueue(parallel=4)
tasks = tc.create_downsampling_tasks(cloudpath)
tq.insert(tasks)
tasks = tc.create_meshing_tasks(cloudpath, mip=0, compress=False)
tq.insert(tasks)
tasks = tc.create_mesh_manifest_tasks(cloudpath)
tq.insert(tasks)
tq.execute(progress=True)


Tasks:   0%|          | 0/1039 [00:00<?, ?it/s]

Volume Bounds:  Bbox([0, 0, 0],[1012, 1365, 447], dtype=int32)
Selected ROI:   Bbox([0, 0, 0],[1012, 1365, 447], dtype=int32)




Downloading:   0%|          | 0/88 [00:00<?, ?it/s]

Downloading:   0%|          | 0/88 [00:00<?, ?it/s]

Downloading:   0%|          | 0/88 [00:00<?, ?it/s]

Downloading:   0%|          | 0/88 [00:00<?, ?it/s]

Downloading:  39%|███▊      | 34/88 [00:00<00:00, 260.84it/s]

Downloading:  52%|█████▏    | 46/88 [00:00<00:00, 264.24it/s]

Downloading:  67%|██████▋   | 59/88 [00:00<00:00, 225.18it/s]

Downloading:  67%|██████▋   | 59/88 [00:00<00:00, 225.18it/s]

Downloading:  72%|███████▏  | 63/88 [00:00<00:00, 225.18it/s]

Downloading:  62%|██████▎   | 55/88 [00:00<00:00, 153.78it/s]

Downloading:  62%|██████▎   | 55/88 [00:00<00:00, 153.78it/s]

Downloading:  41%|████      | 36/88 [00:00<00:00, 207.16it/s]

Downloading:  76%|███████▌  | 67/88 [00:00<00:00, 153.78it/s]

Downloading:  82%|████████▏ | 72/88 [00:00<00:00, 153.78it/s]

Downloading:  99%|█████████▉| 87/88 [00:00<00:00, 120.76it/s]

Downloading:  47%|████▋     | 41/88 [00:00<00:00, 64.01it/s] 

Downloading: 100%|██████████| 

Uploading: 100%|██████████| 24/24 [00:00<00:00, 174.58it/s]

Uploading: 100%|██████████| 24/24 [00:00<00:00, 174.58it/s]

Uploading: 100%|██████████| 24/24 [00:00<00:00, 131.57it/s]

Uploading: 100%|██████████| 24/24 [00:00<00:00, 132.93it/s]

Uploading: 100%|██████████| 24/24 [00:00<00:00, 131.57it/s]

Uploading:  88%|████████▊ | 21/24 [00:00<00:00, 119.57it/s]

Uploading:  92%|█████████▏| 22/24 [00:00<00:00, 119.57it/s]

Uploading: 100%|██████████| 24/24 [00:00<00:00, 119.57it/s]

Uploading: 100%|██████████| 24/24 [00:00<00:00, 119.57it/s]

Uploading: 100%|██████████| 24/24 [00:00<00:00, 119.57it/s]

Uploading: 100%|██████████| 24/24 [00:00<00:00, 119.57it/s]

Uploading: 100%|██████████| 24/24 [00:00<00:00, 119.57it/s]

Uploading: 100%|██████████| 24/24 [00:00<00:00, 119.57it/s]

Uploading: 100%|██████████| 24/24 [00:00<00:00, 119.57it/s]

Uploading: 100%|██████████| 24/24 [00:00<00:00, 119.57it/s]

Uploading: 100%|██████████| 24/24 [00:00<00:00, 119.57it/s]

Uploading: 100%|████████

Downloading: 100%|██████████| 88/88 [00:00<00:00, 42.67it/s]

Downloading: 100%|██████████| 88/88 [00:00<00:00, 229.18it/s]

Downloading: 100%|██████████| 88/88 [00:00<00:00, 229.18it/s]

Downloading: 100%|██████████| 88/88 [00:00<00:00, 42.67it/s]

Downloading: 100%|██████████| 88/88 [00:00<00:00, 42.67it/s]

Downloading: 100%|██████████| 88/88 [00:00<00:00, 42.67it/s]

Downloading: 100%|██████████| 88/88 [00:00<00:00, 42.67it/s]

Downloading: 100%|██████████| 88/88 [00:00<00:00, 42.67it/s]

Uploading:   0%|          | 0/24 [00:00<?, ?it/s]

Downloading: 100%|██████████| 88/88 [00:00<00:00, 197.45it/s][A


Downloading: 100%|██████████| 88/88 [00:00<00:00, 170.07it/s]


Uploading:  88%|████████▊ | 21/24 [00:00<00:00, 142.24it/s]

Uploading:  96%|█████████▌| 23/24 [00:00<00:00, 142.24it/s]

Uploading: 100%|██████████| 24/24 [00:00<00:00, 142.24it/s]

Uploading: 100%|██████████| 24/24 [00:00<00:00, 142.24it/s]

Uploading: 100%|██████████| 24/24 [00:00<00:00, 142.24it/s]

Uploading: 100%|

...

Downloading: 100%|██████████| 88/88 [00:00<00:00, 226.45it/s]

Downloading: 100%|██████████| 88/88 [00:00<00:00, 371.61it/s]

Downloading: 100%|██████████| 88/88 [00:00<00:00, 226.45it/s]

Downloading: 100%|██████████| 88/88 [00:00<00:00, 226.45it/s]

Downloading: 100%|██████████| 88/88 [00:00<00:00, 226.45it/s]

Downloading: 100%|██████████| 88/88 [00:00<00:00, 371.61it/s]

Downloading: 100%|██████████| 88/88 [00:00<00:00, 419.90it/s]

Downloading: 100%|██████████| 88/88 [00:00<00:00, 419.90it/s]

Downloading: 100%|██████████| 88/88 [00:00<00:00, 419.90it/s]

Downloading: 100%|██████████| 88/88 [00:00<00:00, 419.90it/s]

Downloading: 100%|██████████| 88/88 [00:00<00:00, 419.90it/s]

Downloading: 100%|██████████| 88/88 [00:00<00:00, 371.61it/s]

Downloading: 100%|██████████| 88/88 [00:00<00:00, 371.61it/s]

Downloading: 100%|██████████| 88/88 [00:00<00:00, 371.61it/s]

Downloading: 100%|██████████| 88/88 [00:00<00:00, 371.61it/s]

Downloading: 100%|██████████| 88/88 [00:00<00:00, 226.4

Downloading:  29%|██▉       | 132/448 [00:00<00:00, 390.01it/s]

Downloading:  34%|███▍      | 154/448 [00:00<00:00, 390.01it/s]

Downloading:  38%|███▊      | 170/448 [00:00<00:00, 390.01it/s]

Downloading: 100%|██████████| 112/112 [00:00<00:00, 259.83it/s]

Downloading: 100%|██████████| 112/112 [00:00<00:00, 259.83it/s]

Downloading:  42%|████▏     | 190/448 [00:00<00:00, 390.01it/s]

Downloading: 100%|██████████| 112/112 [00:00<00:00, 259.83it/s]

Downloading:  11%|█         | 61/560 [00:00<00:01, 426.10it/s]

Downloading: 100%|██████████| 112/112 [00:00<00:00, 259.83it/s]

Downloading: 100%|██████████| 112/112 [00:00<00:00, 259.83it/s]

Downloading: 100%|██████████| 112/112 [00:00<00:00, 259.83it/s]

Downloading:  52%|█████▏    | 235/448 [00:00<00:00, 390.01it/s]

Downloading: 100%|██████████| 112/112 [00:00<00:00, 259.83it/s]

Downloading:  13%|█▎        | 74/560 [00:00<00:04, 111.96it/s]

Downloading:  60%|██████    | 271/448 [00:00<00:01, 129.46it/s]

Downloading:  65%|██████▌  

Downloading: 100%|██████████| 560/560 [00:01<00:00, 25.82it/s]

Downloading: 100%|██████████| 560/560 [00:01<00:00, 25.82it/s]

Downloading: 100%|██████████| 560/560 [00:01<00:00, 25.82it/s]

Downloading: 100%|██████████| 560/560 [00:01<00:00, 25.82it/s]

Downloading:  98%|█████████▊| 550/560 [00:01<00:00, 20.74it/s]

Downloading:  98%|█████████▊| 550/560 [00:01<00:00, 20.74it/s]

Downloading:  99%|█████████▉| 553/560 [00:01<00:00, 20.74it/s]

Downloading:  99%|█████████▉| 556/560 [00:01<00:00, 20.74it/s]

Downloading: 100%|██████████| 560/560 [00:01<00:00, 20.40it/s]

Downloading: 100%|██████████| 560/560 [00:01<00:00, 20.40it/s]

Downloading: 100%|██████████| 560/560 [00:01<00:00, 20.40it/s]

Downloading: 100%|██████████| 560/560 [00:01<00:00, 20.40it/s]

Downloading: 100%|██████████| 560/560 [00:01<00:00, 25.82it/s]

Downloading: 100%|██████████| 560/560 [00:01<00:00, 25.82it/s]

Downloading: 100%|██████████| 560/560 [00:01<00:00, 25.82it/s]

Downloading: 100%|██████████| 560/560 [0

Downloading:  92%|█████████▏| 647/700 [00:01<00:04, 11.17it/s]

Downloading:  93%|█████████▎| 652/700 [00:01<00:03, 13.93it/s]

Downloading:  94%|█████████▍| 660/700 [00:01<00:01, 20.47it/s]

Downloading:  95%|█████████▍| 664/700 [00:01<00:01, 20.47it/s]

Downloading:  95%|█████████▌| 668/700 [00:01<00:01, 20.88it/s]

Downloading:  96%|█████████▌| 671/700 [00:01<00:01, 20.88it/s]

Downloading:  96%|█████████▋| 675/700 [00:02<00:01, 17.44it/s]

Downloading:  97%|█████████▋| 677/700 [00:02<00:01, 18.97it/s]

Downloading:  97%|█████████▋| 678/700 [00:02<00:01, 15.87it/s]

Downloading:  97%|█████████▋| 682/700 [00:02<00:01, 12.94it/s]

Downloading:  97%|█████████▋| 682/700 [00:02<00:01, 12.94it/s]

Downloading: 100%|██████████| 448/448 [00:02<00:00, 223.32it/s]


Downloading:  98%|█████████▊| 688/700 [00:02<00:00, 12.71it/s]

Downloading:  99%|█████████▊| 690/700 [00:02<00:00, 13.34it/s]

Downloading:  99%|█████████▊| 691/700 [00:02<00:00, 13.34it/s]

Downloading:  99%|█████████▉| 694/700 

Downloading: 100%|██████████| 560/560 [00:01<00:00,  6.24it/s]

Downloading: 100%|██████████| 560/560 [00:01<00:00,  6.24it/s]

Downloading: 100%|██████████| 560/560 [00:01<00:00,  6.24it/s]

Downloading: 100%|██████████| 560/560 [00:01<00:00,  6.24it/s]

Downloading: 100%|██████████| 560/560 [00:01<00:00,  6.24it/s]

Uploading: 100%|██████████| 2464/2464 [00:30<00:00, 15.32it/s]

Downloading:   0%|          | 0/112 [00:00<?, ?it/s]

Downloading:   1%|          | 1/112 [00:00<00:22,  4.84it/s]

Downloading:  12%|█▏        | 13/112 [00:00<00:14,  6.79it/s]

Downloading:  18%|█▊        | 20/112 [00:00<00:09,  9.20it/s]

Downloading:  29%|██▉       | 33/112 [00:00<00:06, 12.74it/s]

Downloading:  43%|████▎     | 48/112 [00:00<00:03, 17.56it/s]

Downloading:  56%|█████▋    | 63/112 [00:00<00:02, 23.87it/s]

Downloading:  66%|██████▌   | 74/112 [00:00<00:01, 31.07it/s]

Downloading:  77%|███████▋  | 86/112 [00:01<00:00, 32.60it/s]

Downloading:  78%|███████▊  | 87/112 [00:01<00:00, 34.06it/